# Extract, Transform and Load: Postal Codes of Vancouver, BC

In [1]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup

Create a target file in which we will later load the transformed data and a logfile

In [2]:
open("logfile.txt", "w").close()
open("Vancouver_postal_code.csv", "w").close()
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "Vancouver_postal_code.csv"   # file where transformed data is stored

### Scraping Data from Wikipedia

In [3]:
# Scraping Data from Wikipedia
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V")
soup = BeautifulSoup(r.content, "html.parser")

Defining a function to extract the table

In [4]:
def extract():
    table=soup.find('table')
    table_rows = table.find_all('tr')

    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
    extracted_data=pd.DataFrame(l, columns=["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"])
    extracted_data = extracted_data.replace(r'\n',' ', regex=True)

    return extracted_data  

### Transforming Data

We chose a column and split it into 3 columns

In [5]:
# Creating a new column for Neighbourhoods
def transform(data, column):
    dato=pd.DataFrame(columns=['City','PostalCode', 'Neighbourhood'])
    dato["PostalCode"] = data[column].str[:3]
    mu=data[column].str[3:-2].str.split("(", n = 1, expand = True)
    dato["City"] = mu[0]
    dato["Neighbourhood"]= mu[1]
    return dato

### Load

Create a function that takes a dataframe and load it to a csv named Neighbourhoods_Vancouver.csv.

In [6]:
def load(targetfile,data_to_load):
    # Write your code here
    data_to_load.to_csv(targetfile)     

### Logging

Create a function to log the process

In [7]:
def log(message):
    # Write your code here
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')    

## ETL-Process

In [8]:
log("ETL Job Started")
log("Extract phase Started")

In [9]:
#Extract
extracted_data = extract()
extracted_data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9
0,V1AKimberley,V2APenticton,V3ALangley Township(Langley City),V4ASurreySouthwest,V5ABurnaby(Government Road / Lake City / SFU /...,V6AVancouver(Strathcona / Chinatown / Downtown...,V7ARichmondSouth,V8APowell River,V9AVictoria(Vic West / Esquimalt)Canadian Forc...
1,V1BVernonEast,V2BKamloopsNorthwest,V3BPort CoquitlamCentral,V4BWhite Rock,V5BBurnaby(Parkcrest-Aubrey / Ardingley-Sprott),V6BVancouver(NE Downtown / Gastown / Harbour C...,V7BRichmond(Sea Island / YVR),V8BSquamish,V9BVictoria(West Highlands / North Langford / ...
2,V1CCranbrook,V2CKamloopsCentral and Southeast,V3CPort CoquitlamSouth,V4CDeltaNortheast,V5CBurnaby(Burnaby Heights / Willingdon Height...,V6CVancouver(Waterfront / Coal Harbour / Canad...,V7CRichmondNorthwest,V8CKitimat,V9CVictoria(Colwood / South Langford / Metchos...
3,V1ESalmon Arm,V2EKamloopsSouth and West,V3ECoquitlamNorth,V4EDeltaEast,V5EBurnaby(Lakeview-Mayfield / Richmond Park /...,V6EVancouver(SE West End / Davie Village),V7ERichmondSouthwest,V8EWhistler,V9EVictoria(East Highlands / NW Saanich)
4,V1GDawson Creek,V2GWilliams Lake,V3GAbbotsfordEast,V4GDeltaEast Central,V5GBurnaby(Cascade-Schou / Douglas-Gilpin),V6GVancouver(NW West End / Stanley Park),V7GNorth Vancouver (district municipality)Oute...,V8GTerrace,V9GLadysmith


The data for Vancouver is in column "V6"

In [10]:
log("Extract phase Ended")

In [11]:
log("Transform phase Started")

In [12]:
#Transform
transformed_data = transform(extracted_data,"V6")

In [13]:
log("Transform phase Ended")

In [14]:
log("Load phase Started")

In [15]:
#Load
load(targetfile,transformed_data)

In [16]:
log("Load phase Ended")

Result: We extracted the table from Wikipedia, transformed the column containing the data on Vancouver, loaded it into our targetfile and wrote a logfile of the process

In [17]:
pd.read_csv(targetfile, index_col=0)

,City,PostalCode,Neighbourhood
0,Vancouver,V6A,Strathcona / Chinatown / Downtown Eastside
1,Vancouver,V6B,NE Downtown / Gastown / Harbour Centre / Inter...
2,Vancouver,V6C,Waterfront / Coal Harbour / Canada Place
3,Vancouver,V6E,SE West End / Davie Village
4,Vancouver,V6G,NW West End / Stanley Park
5,Vancouver,V6H,West Fairview / Granville Island / NE Shaughnessy
6,Vancouver,V6J,NW Shaughnessy / East Kitsilano / Quilchena
7,Vancouver,V6K,Central Kitsilano / Greektown
8,Vancouver,V6L,NW Arbutus Ridge / NE Dunbar-Southlands
9,Vancouver,V6M,South Shaughnessy / NW Oakridge / NE Kerrisdal...


In [18]:
pd.read_csv(logfile)

,2021-Feb-26-14:31:32,ETL Job Started
0,2021-Feb-26-14:31:32,Extract phase Started
1,2021-Feb-26-14:31:32,Extract phase Ended
2,2021-Feb-26-14:31:32,Transform phase Started
3,2021-Feb-26-14:31:32,Transform phase Ended
4,2021-Feb-26-14:31:32,Load phase Started
5,2021-Feb-26-14:31:32,Load phase Ended
